# =================== Import ===================

In [1]:
import numpy as np
import scipy.stats
from pathlib import Path
import os
%matplotlib inline
import matplotlib.pyplot as plt
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t

# =================== Variables ===================

In [2]:
data = np.load('2.CNN_Model/ER_dataset.npz')
dir_pos_att_values = ('3.DeepLoc_SHAP/Pos_Output_Attention/')
dir_neg_att_values = ('3.DeepLoc_SHAP/Neg_Output_Attention')
boxplot_file = ('3.DeepLoc_SHAP/Plot_Boxplot_PosNeg_NormalizedAttValues.png')
plot_p_values_file = ('3.DeepLoc_SHAP/Plot_p_values_alpha.png')
plot_his_p_values_file = ('3.DeepLoc_SHAP/Plot_his_p_values_alpha.png')

# =================== Functions ===================

In [3]:
def normalizing_pos():
    '''Function to normalize the attention values of the proteins, before only using the last 100aa in the sequence'''
    directory = (dir_pos_att_values)
    pos_att_matrix_raw = np.array([])
    pos_att_matrix = np.array([])
    for file in os.listdir(directory):
        if file.endswith(".txt") and not file.endswith('output.txt'): 
            file = os.path.join(directory, file)
            f = open(file)        
            att_vec = []
            for line in f:
                if not line.startswith('#'):
                    line = line.strip().split()
                    att_vec.append(float(line[2]))
            # The attention values are normalized   
            mean_att = np.mean(att_vec)
            std_att = np.std(att_vec) 
            norm = [(float(i)-mean_att)/std_att for i in att_vec]
            # The last 100 aa are extracted
            norm_100 = norm[-100:]
            norm_100 = np.asarray(norm_100)
            if pos_att_matrix.size == 0:
                pos_att_matrix = np.hstack((pos_att_matrix, norm_100))                    
            else: 
                pos_att_matrix = np.vstack((pos_att_matrix, norm_100))        
        else:
            continue 
    return pos_att_matrix

In [4]:
def normalizing_neg():
    '''Function to normalize the attention values of the proteins, before only using the last 100aa in the sequence'''
    directory = (dir_neg_att_values)
    neg_att_matrix_raw = np.array([])
    neg_att_matrix = np.array([])
    for file in os.listdir(directory):
        if file.endswith(".txt") and not file.endswith('output.txt'): 
            file = os.path.join(directory, file)
            f = open(file)        
            att_vec = []
            for line in f:
                if not line.startswith('#'):
                    line = line.strip().split()
                    att_vec.append(float(line[2]))
            # The attention values are normalized   
            mean_att = np.mean(att_vec)
            std_att = np.std(att_vec) 
            norm = [(float(i)-mean_att)/std_att for i in att_vec]
            # The last 100 aa are extracted
            norm_100 = norm[-100:]
            norm_100 = np.asarray(norm_100)
            if neg_att_matrix.size == 0:
                neg_att_matrix = np.hstack((neg_att_matrix, norm_100))
            else:
                if np.size(norm_100) == 100:
                    neg_att_matrix = np.vstack((neg_att_matrix, norm_100))   
                    #neg_att_matrix = np.vstack((neg_att_matrix, norm_100))        
        else:
            continue 
    return neg_att_matrix


In [10]:
def concatenated_boxplot(pos_att_matrix,neg_att_matrix):
    '''Function to make a concatenated boxplot of the normalized attention values of the last 10 positions'''
    pos_10 = pos_att_matrix[:,-10:]
    neg_10 = neg_att_matrix[:,-10:]


    def draw_plot(data, offset,edge_color, fill_color):
        pos = np.arange(data.shape[1])+offset+1
        bp = ax.boxplot(data, positions= pos, widths=0.40, patch_artist=True, manage_xticks=False, showfliers=False)
        for element in ['boxes', 'whiskers', 'fliers', 'medians', 'caps']:
            plt.setp(bp[element], color=edge_color)
        for patch in bp['boxes']:
            patch.set(facecolor=fill_color)

    fig, ax = plt.subplots()
    draw_plot(pos_10, -0.2, "tomato","red")
    draw_plot(neg_10, +0.2,"lightblue", "skyblue")

    plt.ylabel("Normalized attention value", fontsize=10)
    plt.xlabel("Amino acid position", fontsize=10)
    plt.ylim(-1,4.5)
    plt.xticks(np.arange(0, 11, 1))
    # Adding legends to the plot
    plt.gca().legend(["Positives","Negatives"],loc=2,fontsize = 'medium')
    # Save figure to file
    fig.savefig(boxplot_file,bbox_inches='tight')
    plt.close()
    return fig, pos_10, neg_10

In [15]:
def t_test(pos_10,neg_10):
    '''Function to make a t-test for the independent samples positives and negatives'''
    alpha = 0.05 #95% confidence interval
    t_val = []
    p_val = []
    for i in range(len(pos_10[0,:])):
        # calculate means
        mean1, mean2 = mean(pos_10[:,i]), mean(neg_10[:,i])
        # calculate standard errors
        se1, se2 = sem(pos_10[:,i]), sem(neg_10[:,i])
        # standard error on the difference between the samples
        sed = sqrt(se1**2.0 + se2**2.0)
        # calculate the t statistic
        t_stat = (mean1 - mean2) / sed
        # degrees of freedom
        df = len(pos_10[:,i]) + len(neg_10[:,i]) - 2
        # calculate the critical value
        cv = t.ppf(1.0 - alpha, df) #The critical value is calculated by using the confidence interval
        # calculate the p-value
        p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
        t_val.append(t_stat)
        p_val.append(p)
        print('t=%.3f, df=%d, cv=%.3f, p=%.3f' % (t_stat, df, cv, p))

        # interpret via critical value
        if abs(t_stat) <= cv:
            print('cv: Accept null hypothesis that the means are equal.')
        else:
            print('cv: Reject the null hypothesis that the means are equal.')
        
        # interpret via p-value
        if p > alpha:
            print('p: Accept null hypothesis that the means are equal.')
        else:
            print('p: Reject the null hypothesis that the means are equal.')
    return t_val,p_val,alpha

In [16]:
def p_values_plot(p_val,alpha): 
    '''Function to save plot over p-values in a file'''
    # Plot showing the p-values
    fig_p = plt.figure(figsize=(20,10))
    plt.plot(p_val, linewidth=2.5, color='skyblue')
    plt.hlines(y=alpha, xmin=0, xmax=9, linewidth=2.5, color='red')
    # Adding legends to the plot
    plt.gca().legend(('p-values','Alpha = 0.05'),fontsize = 'large')
    plt.ylabel("p-value", fontsize=15)
    plt.xlabel("Amino acid position", fontsize=15)
    plt.savefig(plot_p_values_file)
    plt.close()
    return fig_p

In [17]:
def p_values_his(p_val,alpha): 
    '''Function to save plot over p-values in a file'''
    if p_val[0] == 0:
        None
    else:
        p_val = np.insert(p_val,0,0)
    his_p_value = plt.figure(figsize=(20,10))
    plt.bar(np.arange(len(p_val)), p_val,color='skyblue')
    plt.hlines(y=alpha, xmin=0.5, xmax=11, linewidth=2.5, color='red')
    # Adding legends to the plot
    plt.gca().legend(('Alpha = 0.05','p-values'),fontsize = 'large')
    plt.xlim([0.5, 9.5])
    plt.xticks(np.arange(1, 12,1))
    plt.ylabel("p-value", fontsize=15)
    plt.xlabel("Amino acid position", fontsize=15)
    #plt.show()
    plt.savefig(plot_his_p_values_file)
    plt.close()
    return his_p_value

# =================== Main ===================

In [18]:
pos_att_matrix = normalizing_pos()
neg_att_matrix = normalizing_neg() #Shape under 2474 proteiner, da der er nogle protein under 100aa
fig,pos_10,neg_10 = concatenated_boxplot(neg_att_matrix,pos_att_matrix)
t_val,p_val,alpha = t_test(pos_10,neg_10)
fig_p = p_values_plot(p_val,alpha)
his_p_value = p_values_his(p_val,alpha)

t=-2.153, df=2226, cv=1.646, p=0.031
cv: Reject the null hypothesis that the means are equal.
p: Reject the null hypothesis that the means are equal.
t=-1.749, df=2226, cv=1.646, p=0.080
cv: Reject the null hypothesis that the means are equal.
p: Accept null hypothesis that the means are equal.
t=-0.544, df=2226, cv=1.646, p=0.587
cv: Accept null hypothesis that the means are equal.
p: Accept null hypothesis that the means are equal.
t=-0.297, df=2226, cv=1.646, p=0.766
cv: Accept null hypothesis that the means are equal.
p: Accept null hypothesis that the means are equal.
t=-0.095, df=2226, cv=1.646, p=0.924
cv: Accept null hypothesis that the means are equal.
p: Accept null hypothesis that the means are equal.
t=1.493, df=2226, cv=1.646, p=0.136
cv: Accept null hypothesis that the means are equal.
p: Accept null hypothesis that the means are equal.
t=-2.224, df=2226, cv=1.646, p=0.026
cv: Reject the null hypothesis that the means are equal.
p: Reject the null hypothesis that the mean